In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import torch
from dataset import Model10NetDataset
from model import TNet, PointNet

In [3]:
torch.cuda.is_available()

True

In [4]:
dataset = Model10NetDataset(path="./ModelNet10")

In [5]:
point_cloud = dataset[0][0]
point_cloud.shape

torch.Size([3, 1601])

In [6]:
point_cloud = point_cloud.to("cuda")
point_cloud.shape

torch.Size([3, 1601])

In [7]:
def stack(x):
    # Add a dimension to make it (1, N, C)
    x_unsqueezed = x.unsqueeze(0)

    # Concatenate along the new dimension to make it (2, N, C)
    x_stacked = torch.cat([x_unsqueezed, x_unsqueezed], dim=0)

    return x_stacked

In [8]:
point_cloud = stack(point_cloud)
point_cloud.shape

torch.Size([2, 3, 1601])

In [9]:
tnet = TNet(in_features=3)
tnet = tnet.to("cuda")
tnet

TNet(
  (mlp): Sequential(
    (0): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
    (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
    (4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
    (7): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU()
  )
  (fc): Sequential(
    (0): Linear(in_features=1024, out_features=512, bias=True)
    (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): Linear(in_features=512, out_features=256, bias=True)
    (4): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU()
    (6): Linear(in_features=256, out_features=9, bias=True)
  )
  (batchnorm): BatchNorm1d(3, eps=1e-05, momentum=0.1, affine=

In [10]:
out = tnet(point_cloud)
out.shape

torch.Size([2, 3, 3])

In [ ]:
out

tensor([[[0.0000, 1.2749, 0.0000,  ..., 0.0000, 0.2662, 1.9018],
         [0.0000, 0.9713, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.5344, 0.0000, 0.6598,  ..., 0.0000, 0.3981, 1.0012],
         ...,
         [0.9243, 0.0000, 0.0000,  ..., 0.2911, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.5123,  ..., 0.7256, 0.0000, 0.0000],
         [0.6774, 0.4034, 0.0000,  ..., 0.0000, 0.8900, 0.4463]],

        [[0.0000, 1.2749, 0.0000,  ..., 0.0000, 0.2662, 1.9018],
         [0.0000, 0.9713, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.5344, 0.0000, 0.6598,  ..., 0.0000, 0.3981, 1.0012],
         ...,
         [0.9243, 0.0000, 0.0000,  ..., 0.2911, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.5123,  ..., 0.7256, 0.0000, 0.0000],
         [0.6774, 0.4034, 0.0000,  ..., 0.0000, 0.8900, 0.4463]]],
       device='cuda:0', grad_fn=<ReluBackward0>)

In [11]:
pointnet = PointNet()
pointnet = pointnet.to("cuda")
pointnet

PointNet(
  (input_transform): TNet(
    (mlp): Sequential(
      (0): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
      (1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
      (4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
      (6): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
      (7): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (8): ReLU()
    )
    (fc): Sequential(
      (0): Linear(in_features=1024, out_features=512, bias=True)
      (1): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): Linear(in_features=512, out_features=256, bias=True)
      (4): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU()
      (6): Linear(in_features=256, out_features=9, bias=Tru

In [13]:
x = pointnet(point_cloud)
x[0].shape, x[1].shape, x[2].shape

(torch.Size([2, 10]), torch.Size([2, 3, 3]), torch.Size([2, 64, 64]))